In this notebook we will experiment with store sales data

In [1]:
%autosave 300
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/demand_forecasting_kernels"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/demand_forecasting_kernels


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [4]:
train_df = pd.read_csv(r"data/demand-forecasting-kernels-only/train.csv")
test_df = pd.read_csv(r"data/demand-forecasting-kernels-only/test.csv")

In [5]:
print("Store level summary")
print(
    train_df.groupby(["store"], as_index=False).agg({"date": ["min", "max", "nunique"]})
)
print(
    test_df.groupby(["store"], as_index=False).agg({"date": ["min", "max", "nunique"]})
)

Store level summary
  store        date                    
                min         max nunique
0     1  2013-01-01  2017-12-31    1826
1     2  2013-01-01  2017-12-31    1826
2     3  2013-01-01  2017-12-31    1826
3     4  2013-01-01  2017-12-31    1826
4     5  2013-01-01  2017-12-31    1826
5     6  2013-01-01  2017-12-31    1826
6     7  2013-01-01  2017-12-31    1826
7     8  2013-01-01  2017-12-31    1826
8     9  2013-01-01  2017-12-31    1826
9    10  2013-01-01  2017-12-31    1826
  store        date                    
                min         max nunique
0     1  2018-01-01  2018-03-31      90
1     2  2018-01-01  2018-03-31      90
2     3  2018-01-01  2018-03-31      90
3     4  2018-01-01  2018-03-31      90
4     5  2018-01-01  2018-03-31      90
5     6  2018-01-01  2018-03-31      90
6     7  2018-01-01  2018-03-31      90
7     8  2018-01-01  2018-03-31      90
8     9  2018-01-01  2018-03-31      90
9    10  2018-01-01  2018-03-31      90


In [6]:
print("Item level summary")
print(train_df.groupby(["item"], as_index=False).agg({"date": ["min", "max", "nunique"]}))
print(test_df.groupby(["item"], as_index=False).agg({"date": ["min", "max", "nunique"]}))

Item level summary
   item        date                    
                min         max nunique
0     1  2013-01-01  2017-12-31    1826
1     2  2013-01-01  2017-12-31    1826
2     3  2013-01-01  2017-12-31    1826
3     4  2013-01-01  2017-12-31    1826
4     5  2013-01-01  2017-12-31    1826
5     6  2013-01-01  2017-12-31    1826
6     7  2013-01-01  2017-12-31    1826
7     8  2013-01-01  2017-12-31    1826
8     9  2013-01-01  2017-12-31    1826
9    10  2013-01-01  2017-12-31    1826
10   11  2013-01-01  2017-12-31    1826
11   12  2013-01-01  2017-12-31    1826
12   13  2013-01-01  2017-12-31    1826
13   14  2013-01-01  2017-12-31    1826
14   15  2013-01-01  2017-12-31    1826
15   16  2013-01-01  2017-12-31    1826
16   17  2013-01-01  2017-12-31    1826
17   18  2013-01-01  2017-12-31    1826
18   19  2013-01-01  2017-12-31    1826
19   20  2013-01-01  2017-12-31    1826
20   21  2013-01-01  2017-12-31    1826
21   22  2013-01-01  2017-12-31    1826
22   23  2013-01-01  

##### basic pre-processing

In [7]:
full_df = pd.concat([train_df, test_df], axis=0)
print(full_df.shape)

(958000, 5)


In [8]:
full_df["date"] = pd.to_datetime(full_df["date"], format="%Y-%m-%d", errors="coerce")
full_df["store"] = full_df["store"].astype("object")
full_df["item"] = full_df["item"].astype("object")
full_df["sales"] = full_df["sales"].astype(np.float32)

In [9]:
full_df = full_df.drop(['id'],axis=1)

In [10]:
print(full_df.isnull().sum())

date         0
store        0
item         0
sales    45000
dtype: int64


#### Creating basic date features -- Daily data

In [21]:
full_df.head()

,date,store,item,sales,year,month,day_of_the_year,week_of_the_year,day_of_the_week,is_weekend,is_month_start,is_month_end
0,2013-01-01,1,1,13.0,2013,1,1,1,1,0,1,0
1,2013-01-02,1,1,11.0,2013,1,2,1,2,0,0,0
2,2013-01-03,1,1,14.0,2013,1,3,1,3,0,0,0
3,2013-01-04,1,1,13.0,2013,1,4,1,4,1,0,0
4,2013-01-05,1,1,10.0,2013,1,5,1,5,1,0,0


In [14]:
full_df["year"] = full_df["date"].dt.year
full_df["month"] = full_df["date"].dt.month
full_df["day_of_the_year"] = full_df["date"].dt.dayofyear
full_df["week_of_the_year"] = full_df["date"].dt.isocalendar().week
full_df["day_of_the_week"] = full_df["date"].dt.dayofweek

In [20]:
# 0-Monday and so on
full_df["is_weekend"] = full_df["date"].dt.dayofweek // 4
full_df["is_month_start"] = full_df["date"].dt.is_month_start.astype(int)
full_df["is_month_end"] = full_df["date"].dt.is_month_end.astype(int)

In [31]:
full_df['quarter'] = full_df['date'].dt.quarter

In [22]:
full_df["week_of_the_year"].max(), full_df["day_of_the_year"].max()

(53, 366)

#### Cyclical features

In [23]:
from sklearn.preprocessing import FunctionTransformer

In [27]:
def sin_transformer(period):
    return  lambda x: np.sin(x / period * 2 * np.pi)


def cos_transformer(period):
    return  lambda x: np.cos(x / period * 2 * np.pi)

In [32]:
full_df["month_sin"] = full_df["month"].apply(sin_transformer(12))
full_df["month_cos"] = full_df["month"].apply(cos_transformer(12))

full_df["day_sin"] = full_df["day_of_the_year"].apply(sin_transformer(365))
full_df["day_cos"] = full_df["day_of_the_year"].apply(cos_transformer(365))

full_df["week_sin"] = full_df["week_of_the_year"].apply(sin_transformer(52))
full_df["week_cos"] = full_df["week_of_the_year"].apply(cos_transformer(52))

full_df["quarter_sin"] = full_df["quarter"].apply(sin_transformer(4))
full_df["quarter_cos"] = full_df["quarter"].apply(cos_transformer(4))

In [33]:
full_df.head()

,date,store,item,sales,year,month,day_of_the_year,week_of_the_year,day_of_the_week,is_weekend,is_month_start,is_month_end,month_sin,month_cos,day_sin,day_cos,week_sin,week_cos,quarter,quarter_sin,quarter_cos
0,2013-01-01,1,1,13.0,2013,1,1,1,1,0,1,0,0.5,0.866025,0.017213,0.999852,0.120537,0.992709,1,1.0,6.123234e-17
1,2013-01-02,1,1,11.0,2013,1,2,1,2,0,0,0,0.5,0.866025,0.034422,0.999407,0.120537,0.992709,1,1.0,6.123234e-17
2,2013-01-03,1,1,14.0,2013,1,3,1,3,0,0,0,0.5,0.866025,0.051620,0.998667,0.120537,0.992709,1,1.0,6.123234e-17
3,2013-01-04,1,1,13.0,2013,1,4,1,4,1,0,0,0.5,0.866025,0.068802,0.997630,0.120537,0.992709,1,1.0,6.123234e-17
4,2013-01-05,1,1,10.0,2013,1,5,1,5,1,0,0,0.5,0.866025,0.085965,0.996298,0.120537,0.992709,1,1.0,6.123234e-17


#### Feature Engineering

In [ ]:
full_df = full_df.sort_values(["store", "item", "date"], axis=0)